In [ ]:
#Junko
import os
import sys
import pathlib
import csv

import numpy as np
import tensorflow as tf

import matplotlib.pyplot as plt

tf.get_logger().setLevel('WARNING') # https://stackoverflow.com/a/38873777

from itertools import permutations
import random

from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
pathToData = "/content/drive/MyDrive/Colab Notebooks/data/plans" #text files
output = "/content/drive/MyDrive/Colab Notebooks/data/out.txt" #amalgamated text file
csvPath = "/content/drive/MyDrive/Colab Notebooks/data/data.csv"

In [ ]:
data = []
os.makedirs(pathToData, exist_ok=True)
#https://www.geeksforgeeks.org/reading-rows-from-a-csv-file-in-python/
with open(csvPath, 'r') as csvFile:
    reader = csv.reader(csvFile)
    next(reader)
    for index, row in enumerate(reader, start=1):
        #https://www.w3schools.com/python/ref_string_strip.asp
        addictionName = row[1].strip()
        triggers = [row[2].strip().lower(), row[3].strip().lower(), row[4].strip().lower()]
        warningSigns = [row[5].strip().lower(), row[6].strip().lower(), row[7].strip().lower()]
        copingStrategies = [row[8].strip().lower(), row[9].strip().lower(), row[10].strip().lower()]
        severity = float(row[11].strip())
        #https://stackoverflow.com/questions/104420/how-do-i-generate-all-permutations-of-a-list
        triggerPerm = list(permutations(triggers))
        warningPerm = list(permutations(warningSigns))
        strategyPerm = list(permutations(copingStrategies))
        plan = []
        for i in range(1, 11):
            for trigger in triggerPerm:
                for warning in warningPerm:
                    for strategy in strategyPerm:
                        plan.append("Addiction name: " + addictionName + "\n")
                        plan.append("Triggers:\n")
                        plan.append("- " + trigger[0] + "\n")
                        plan.append("- " + trigger[1] + "\n")
                        plan.append("- " + trigger[2] + "\n")
                        plan.append("Addiction Severity:\n")
                        plan.append(str(float(i / 10)) + "\n")
                        plan.append("Warning Signs:\n")
                        plan.append("- " + warning[0] + "\n")
                        plan.append("- " + warning[1] + "\n")
                        plan.append("- " + warning[2] + "\n")
                        plan.append("Coping Strategies:\n")
                        plan.append("- " + strategy[0] + "\n")
                        plan.append("- " + strategy[1] + "\n")
                        plan.append("- " + strategy[2] + "\n")
                        #print(plan)
                        data.append(''.join(plan))
                        plan = []
        print(data)
for i in range(len(data)):
    formattedPlan = ''
    currentPlan = data[i]
    splitString = currentPlan.split("\n")
    for i in range(len(splitString)):
        formattedPlan = formattedPlan + str(splitString[i].strip())
    addictionName = formattedPlan[0].replace('Addiction name: ', '')

    file_name = f"{addictionName.replace(' ', '_')}_plan_{index}{i}.txt"
    file_path = os.path.join(pathToData, file_name)

    with open(file_path, 'w') as txt_file:
        txt_file.write(formattedPlan.strip())

print(pathToData)


In [ ]:
#https://stackoverflow.com/questions/13613336/how-do-i-concatenate-text-files-in-python
#https://www.geeksforgeeks.org/python-os-listdir-method/
with open(output, 'w') as outfile:
    for fname in os.listdir(pathToData):
        with open(pathToData + "/" + fname) as infile:
            for line in infile:
                outfile.write(line)

In [ ]:
with open(output, 'rb') as i: #https://stackoverflow.com/questions/15746954/what-is-the-difference-between-rb-and-rb-modes-in-file-objects
    text = i.read().decode(encoding='utf-8') #https://www.tutorialspoint.com/python/string_decode.htm
print(repr(text))

In [ ]:
#https://www.geeksforgeeks.org/string-tensors-in-tensorflow/
#https://www.tensorflow.org/api_docs/python/tf/strings/split
def splitLines(plans):
    return tf.strings.split(plans, sep="\n")

In [ ]:
#below code modified from Jeremie Wenger - Artifical Intelligence: https://drive.google.com/file/d/14WFNX1nfbdqhhg0nzr8aFzqdyIriYsa_/view
text_vectorization = tf.keras.layers.TextVectorization( # note that we don't limit our vocab size here (no need)!
    standardize=None,
    split=splitLines, #https://www.tensorflow.org/api_docs/python/tf/keras/layers/TextVectorization
    output_mode="int"
)

text_vectorization.adapt([text])

TOKEN_INDEX = dict(enumerate(text_vectorization.get_vocabulary()))
VOCAB_SIZE = len(text_vectorization.get_vocabulary())   # retrieve the vocab size afterwards

In [ ]:
lm_dataset_raw = tf.data.Dataset.from_tensor_slices([text])

for t in lm_dataset_raw:
    print(t.numpy()[:1000])

b'Addiction name: Gambling\n  Triggers:\n  - lots of money\n  - raffles\n  - lottery\n  Addiction Severity:\n  0.1\n  Warning Signs:\n  - wondering how to use excess cash\n  - lingering around lottery stands\n  - browsing raffle sites\n  Coping Strategies:\n  - block gambling sites\n  - mindfulness\n  - limit access to excess fundsAddiction name: Gambling\n  Triggers:\n  - lots of money\n  - raffles\n  - lottery\n  Addiction Severity:\n  0.1\n  Warning Signs:\n  - lingering around lottery stands\n  - browsing raffle sites\n  - wondering how to use excess cash\n  Coping Strategies:\n  - limit access to excess funds\n  - mindfulness\n  - block gambling sitesAddiction name: Gambling\n  Triggers:\n  - lottery\n  - raffles\n  - lots of money\n  Addiction Severity:\n  0.1\n  Warning Signs:\n  - wondering how to use excess cash\n  - lingering around lottery stands\n  - browsing raffle sites\n  Coping Strategies:\n  - limit access to excess funds\n  - mindfulness\n  - block gambling sitesAddic

In [ ]:
lm_dataset_tok = lm_dataset_raw.map(text_vectorization)

for t in lm_dataset_tok:
    print(t)
    DATASET_LENGTH = t.shape[0]

tf.Tensor([305   3  89 ... 189   7  23], shape=(167072,), dtype=int64)


In [ ]:
# Flatten the dataset of one long tensor into a dataset of individual tokens (ChatGPT 4 12/23)
lm_dataset_flat = lm_dataset_tok.flat_map(
    lambda x: tf.data.Dataset.from_tensor_slices(x)
)
for t in lm_dataset_flat.take(2):
    print(t)

tf.Tensor(305, shape=(), dtype=int64)
tf.Tensor(3, shape=(), dtype=int64)


In [ ]:
SEQUENCE_LENGTH = 3

lm_dataset_seqs = lm_dataset_flat.batch(
    SEQUENCE_LENGTH + 1,
    drop_remainder=True
)

for t in lm_dataset_seqs.take(1):
    print(t)

tf.Tensor([305   3  89  76], shape=(4,), dtype=int64)


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

lm_dataset_batched = (
    lm_dataset_seqs
        .repeat()
        .shuffle(BUFFER_SIZE)
        .batch(BATCH_SIZE, drop_remainder=True)
        .prefetch(tf.data.experimental.AUTOTUNE)
)

for t in lm_dataset_batched.take(1):
    print(t.shape)

(64, 4)


In [ ]:
def prepare_lm_dataset(tokens_batch):
    x = tokens_batch[:, :-1]  # [a b c d e f g] the model predicts top to bottom,
    y = tokens_batch[:, 1:]   # [b c d e f g h] a → b, a b → c, a b c → d, ..., in one go!
    return x, y

lm_dataset = lm_dataset_batched.map(prepare_lm_dataset, num_parallel_calls=4)

In [ ]:
@tf.keras.utils.register_keras_serializable("positional_embedding")
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim

    # new in Keras 3, see: https://keras.io/guides/making_new_layers_and_models_via_subclassing/#best-practice-deferring-weight-creation-until-the-shape-of-the-inputs-is-known
    # See also here: https://github.com/keras-team/keras-nlp/blob/4601d88a61a5d3d15279865769af5155804dd785/keras_nlp/src/layers/modeling/token_and_position_embedding.py#L115
    def build(self, input_shape):
        # token embeddings: semantic information
        self.token_embeddings =tf.keras.layers.Embedding(
            input_dim=self.input_dim, output_dim=self.output_dim
        )
        # position embeddings: syntactic (spatial/temporal) information
        self.position_embeddings =tf.keras.layers.Embedding(
            input_dim=self.sequence_length, output_dim=self.output_dim
        )

    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        embedded_tokens = self.token_embeddings(inputs)
        positions = tf.range(start=0, limit=length, delta=1) # delta: step size
        embedded_positions = self.position_embeddings(positions)
        # both embeddings are simply added together!
        return embedded_tokens + embedded_positions

    # copied from the source here: https://github.com/keras-team/keras-nlp/blob/4601d88a61a5d3d15279865769af5155804dd785/keras_nlp/src/layers/modeling/token_and_position_embedding.py#L146
    def compute_mask(self, inputs, mask=None):
        return self.token_embeddings.compute_mask(inputs, mask=mask)

    def get_config(self): # retrieve config as a dict
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [ ]:
def get_causal_attention_mask(inputs):
    print("Inputs:")
    print(inputs)
    print()
    input_shape = tf.shape(inputs)
    batch_size, sequence_length = input_shape[0], input_shape[1]
    i = tf.range(sequence_length)[:, tf.newaxis]
    j = tf.range(sequence_length)
    print(f"i:\n{i}")
    print()
    print(f"j:\n{j}")
    print()
    mask = tf.cast(i >= j, dtype="int32")
    print("Is i >= j? Boolean cast to ints. (Note the broadcasting)")
    print()
    print(mask)
    print()
    mask = tf.reshape(mask, (1, input_shape[1], input_shape[1])) # adding a batch dimension
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1),
         tf.constant([1, 1], dtype=tf.int32)], axis=0)
    print("We want mask to have the same dims as input, using `tf.tile`.")
    print("Creating the right multiplier for it:")
    print()
    print(mult)
    print()
    tile = tf.tile(mask, mult)
    print("Final mask with batch dimensions:")
    print()
    print(tile)
    return tile

mask = get_causal_attention_mask(tf.random.uniform(shape=(2,10), maxval=50, dtype=tf.int32))

Inputs:
tf.Tensor(
[[27 43 25 28  1 47  1 12 11 44]
 [42 14 12 27 10 41 15 15 46 34]], shape=(2, 10), dtype=int32)

i:
[[0]
 [1]
 [2]
 [3]
 [4]
 [5]
 [6]
 [7]
 [8]
 [9]]

j:
[0 1 2 3 4 5 6 7 8 9]

Is i >= j? Boolean cast to ints. (Note the broadcasting)

tf.Tensor(
[[1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 0 0]
 [1 1 1 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 0 0]
 [1 1 1 1 1 0 0 0 0 0]
 [1 1 1 1 1 1 0 0 0 0]
 [1 1 1 1 1 1 1 0 0 0]
 [1 1 1 1 1 1 1 1 0 0]
 [1 1 1 1 1 1 1 1 1 0]
 [1 1 1 1 1 1 1 1 1 1]], shape=(10, 10), dtype=int32)

We want mask to have the same dims as input, using `tf.tile`.
Creating the right multiplier for it:

tf.Tensor([2 1 1], shape=(3,), dtype=int32)

Final mask with batch dimensions:

tf.Tensor(
[[[1 0 0 0 0 0 0 0 0 0]
  [1 1 0 0 0 0 0 0 0 0]
  [1 1 1 0 0 0 0 0 0 0]
  [1 1 1 1 0 0 0 0 0 0]
  [1 1 1 1 1 0 0 0 0 0]
  [1 1 1 1 1 1 0 0 0 0]
  [1 1 1 1 1 1 1 0 0 0]
  [1 1 1 1 1 1 1 1 0 0]
  [1 1 1 1 1 1 1 1 1 0]
  [1 1 1 1 1 1 1 1 1 1]]

 [[1 0 0 0 0 0 0 0 0 0]
  [1 1 0 0 0 0

In [ ]:
@tf.keras.utils.register_keras_serializable("transformer_decoder")
class TransformerDecoder(tf.keras.layers.Layer):

    # simplified class: we don't need two attention layers as we don't have data
    # flowing from an encoder!

    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim                              # parameters
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.supports_masking = True                            # MASK: enforcing causality

    # new in Keras 3, see: https://keras.io/guides/making_new_layers_and_models_via_subclassing/#best-practice-deferring-weight-creation-until-the-shape-of-the-inputs-is-known
    def build(self, input_shape):
        self.attention_1 = tf.keras.layers.MultiHeadAttention(  # multi-head attention
            num_heads=self.num_heads, key_dim=self.embed_dim
        )
        self.dense_proj = tf.keras.Sequential(                  # dense layer on top: like a nonlinearity
            [tf.keras.layers.Dense(self.dense_dim, activation="relu"),
             tf.keras.layers.Dense(self.embed_dim),
             tf.keras.layers.Dropout(0.1)]
        )
        self.layernorm_1 = tf.keras.layers.LayerNormalization() # layer norm
        self.layernorm_2 = tf.keras.layers.LayerNormalization()

    # retrieve config as a dict (necessary for custom Keras layers)
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, mask=None):

        # prepare the causal mask
        causal_mask = self.get_causal_attention_mask(inputs)

        # REGULAR MASKED ATTENTION
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask) # apply the causal mask

        # residual / layer norm
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)

        # dense net (nonlinearity) / layer norm
        proj_output = self.layernorm_2(self.dense_proj(attention_output_1))

        # residual
        return attention_output_1 + proj_output

In [ ]:
EMBED_DIM = 256
LATENT_DIM = 256
NUM_HEADS = 2
NUM_LAYERS = 5

LEARNING_RATE = 0.001
print(f"learning rate: {LEARNING_RATE}")

def build_model(embed_dim, latent_dim, num_heads, num_layers):
    inputs = tf.keras.Input(shape=(None,), dtype="int64")
    x = PositionalEmbedding(SEQUENCE_LENGTH, VOCAB_SIZE, embed_dim)(inputs)
    for i in range(num_layers):
        x = TransformerDecoder(embed_dim, latent_dim, num_heads)(inputs=x) # no encoder input!
    #x = tf.keras.layers.Dense(VOCAB_SIZE)(x)
    outputs =tf.keras.layers.Dense(VOCAB_SIZE, activation="softmax")(x)    # probability distribution over the vocab
    model = tf.keras.Model(inputs, outputs)
    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=tf.keras.optimizers.RMSprop(LEARNING_RATE),
    )
    return model

model = build_model(EMBED_DIM, LATENT_DIM, NUM_HEADS, NUM_LAYERS)

learning rate: 0.001


In [ ]:
model.summary()

Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)           │ (None, None)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ positional_embedding_1               │ (None, None, 256)           │         140,800 │
│ (PositionalEmbedding)                │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_decoder_5                │ (None, None, 256)           │         658,688 │
│ (TransformerDecoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_decoder_6                │ (None, None, 256)           │         658,688 │
│ (TransformerDecoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_decoder_7                │ (None, None, 256)           │         658,688 │
│ (TransformerDecoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_decoder_8                │ (None, None, 256)           │         658,688 │
│ (TransformerDecoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ transformer_decoder_9                │ (None, None, 256)           │         658,688 │
│ (TransformerDecoder)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_21 (Dense)                     │ (None, None, 547)           │         140,579 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,574,819 (13.64 MB)

 Trainable params: 3,574,819 (13.64 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
def sample_next(predictions, temperature=1.0):
    predictions = np.asarray(predictions).astype("float64")
    predictions = np.log(predictions) / temperature                 # temperature reweighting
    exp_preds = np.exp(predictions)                                 # these two lines are actually
    predictions = exp_preds / np.sum(exp_preds)                     # a softmax
    probas = np.random.multinomial(1, predictions, 1)               # sampling using our probabilities
    return np.argmax(probas)

class TextGenerator(tf.keras.callbacks.Callback):
    def __init__(self,
                 prompt,                                            # initial context
                 generate_length,                                   # how many words to generate
                 seq_length,
                 temperatures=(1.,),                                # a range of different temperatures
                 print_every=50):
        self.prompt = prompt
        self.generate_length = generate_length
        self.seq_length = seq_length
        self.temperatures = temperatures
        self.print_every = print_every

    def on_epoch_end(self, epoch, logs=None):
        if epoch == 0 or (epoch + 1) % self.print_every == 0:
            print()
            print()
            print("EPOCH", epoch + 1)
            print()
            print("-" * 40)
            for temperature in self.temperatures:
                msg = f"temperature {temperature}"
                print(msg)
                print("-" * len(msg))
                # TODO: write the optimised version that does not convert back and forth
                #       from tokens to strings during generation (the current version could
                #       however be used to print the string as it is being generated, with a
                #       'typerwriter effect'...)
                sentence = self.prompt                                                      # start with our prompt
                for i in range(self.generate_length):
                    tokenized_sentence = text_vectorization([sentence])                     # encode the sentence & feed to the model
                    predictions = self.model(tokenized_sentence[:, - self.seq_length + 1:]) # which gives us predictions (crop to seq_len!)
                    next_token = sample_next(predictions[0, -1, :])                         # use these to sample (get the index)
                    sampled_token = TOKEN_INDEX[next_token]                                 # use the index to pick the token
                    sentence += sampled_token + "\n"                                            # add it to our sentence
                print(sentence)
                print()
            print("-" * 40)

# used when debugging
class DatasetInspectionCallback(tf.keras.callbacks.Callback):
    def __init__(self, dataset):
        super(DatasetInspectionCallback, self).__init__()
        self.dataset = dataset

    def on_epoch_begin(self, epoch, logs=None):
        print()
        print(f"\nStarting Epoch {epoch + 1}")
        for i, batch in enumerate(self.dataset.take(1)):
            print(f"Batch {i + 1}:")
            b1, b2 = batch
            print(b1[0, ])
        print()
        print()

# Use the callback in model.fit()
inspection_callback = DatasetInspectionCallback(lm_dataset)

prompt = "Gambling\nTriggers:\n- Boredom on weekends\n- Being near a casino\n- Financial stress\nAddiction Severity:\n0.9\nWarning Signs:\n"
text_gen_callback = TextGenerator(
    prompt,
    generate_length=SEQUENCE_LENGTH,
    seq_length=SEQUENCE_LENGTH,
    temperatures=(0.5, 0.7, 1., 0.9), # a diverse range of temperature to see its effect
    print_every=20,
)

MODELS_DIR = pathlib.Path("models")
MODELS_DIR.mkdir(exist_ok=True)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(
    str(MODELS_DIR / "tiny-gpt.keras"),
    monitor="loss",
    save_best_only=True,
)

print(MODELS_DIR)

models


In [ ]:
EPOCHS = 100
print(DATASET_LENGTH)
print(SEQUENCE_LENGTH)
print(DATASET_LENGTH // (SEQUENCE_LENGTH + 1))
print(BATCH_SIZE)
print(DATASET_LENGTH // (SEQUENCE_LENGTH + 1) // BATCH_SIZE)

167072
3
41768
64
652


In [ ]:
%pip install tensorflowjs
import tensorflowjs as tfjs

model.fit(
    lm_dataset,
    epochs=EPOCHS,
    steps_per_epoch=DATASET_LENGTH // (SEQUENCE_LENGTH + 1) // BATCH_SIZE,
    callbacks=[text_gen_callback, ckpt_callback]
)
tfjs.converters.save_keras_model(model, 'models/junko')

In [ ]:
#prompt = "Gambling\nTriggers:\n- Boredom on weekends\n- Being near a casino\n- Financial stress\nAddiction Severity:\n0.9\nWarning Signs:\n- losing interest in hobbies\n- tiredness\n- biting nails\nCoping Strategies:\n"
#prompt = "Addiction name: Gambling\nTriggers:\n- Boredom on weekends\n- Being near a casino\n- Financial stress\nAddiction Severity:\n0.9\nWarning Signs:\n"
prompt = "Addiction name: Pornography\nTriggers:\n- loneliness\n- boredom\n- tiredness\nAddiction Severity:\n0.6\nWarning Signs:\n"

In [ ]:
RELOAD = True
if RELOAD:
    model = tf.keras.models.load_model(MODELS_DIR / "tiny-gpt.keras") # to load a saved model

def generate(sentence=" ", generate_length=100, temperature=1.):
    # TODO: write the optimised version that does not convert back and forth
    #       from tokens to strings during generation (the current version could
    #       however be used to print the string as it is being generated, with a
    #       'typerwriter effect'...)
    for i in range(generate_length):
        tokenized_sentence = text_vectorization([sentence])                         # encode the sentence & feed to the model
        predictions = model(tokenized_sentence[:, - SEQUENCE_LENGTH + 1:])          # which gives us predictions  (crop to seq_len!)
        next_token = sample_next(predictions[0, -1, :], temperature)                # use these to sample (get the index)
        sampled_token = TOKEN_INDEX[next_token]                                     # use the index to pick the token
        sentence += sampled_token                                                   # add it to our sentence
        sentence += os.linesep
    print(sentence)

def generate_warning_signs(sentence, generate_length, temperature):
  #print(sentence)
  empty = ""
  for i in range(generate_length):
    tokenized_sentence = text_vectorization([sentence])                         # encode the sentence & feed to the model
    predictions = model(tokenized_sentence[:, - SEQUENCE_LENGTH + 1:])          # which gives us predictions  (crop to seq_len!)
    next_token = sample_next(predictions[0, -1, :], temperature)                # use these to sample (get the index)
    sampled_token = TOKEN_INDEX[next_token]                                     # use the index to pick the token
    empty += sampled_token + "\n"
                                                 # add it to our sentence
  #print(sentence)
  return empty

prompt += generate_warning_signs(prompt, 3, .9)
prompt += "Coping Strategies:\n"
prompt += generate_warning_signs(prompt, 3, .9)
#prompt += "- " + generate_warning_sign(sentence=prompt, generate_length=10, temperature=.9)
#prompt += "- " + generate_warning_sign(sentence=prompt, generate_length=10, temperature=.9)
#prompt += "- " + generate_warning_sign(sentence=prompt, generate_length=10, temperature=.9)
print(prompt)

model.save("models/tiny-gpt.h5")

Addiction name: Pornography
Triggers:
- loneliness
- boredom
- tiredness
Addiction Severity:
0.6
Warning Signs:
- peer pressure
- hiding transactions
- secrecy
Coping Strategies:
Addiction Severity:
Addiction Severity:
Addiction Severity:
Warning Signs:
Warning Signs:
  - focus on rebuilding and maintaining healthy relationshipsAddiction name: Gambling
Coping Strategies:
Addiction Severity:
Addiction Severity:
- realise the house always wins
Warning Signs:
Warning Signs:
Warning Signs:
Coping Strategies:
Addiction Severity:
Warning Signs:
Warning Signs:
- being enticed by free credits
- having disposable income
- being enticed by free credits
Coping Strategies:
- realise the house always wins
Warning Signs:
Addiction Severity:



In [ ]:
!tensorflowjs_converter --input_format keras \models/tiny-gpt.h5 \models/junko

2025-02-24 14:12:43.037374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740406363.070853    9740 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740406363.081166    9740 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
🌲 Try https://ydf.readthedocs.io, the successor of TensorFlow Decision Forests with more features and faster training!
failed to lookup keras version from the file,
    this is likely a weight only file


In [ ]:
import json

#https://medium.com/@andrew.w.sale/deploying-text-classification-from-keras-to-tensorflow-js-92c614dca4ec
with open('TOKEN_INDEX.json', 'w') as file:
  json.dump(TOKEN_INDEX, file)